In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Conv2D, Input, AveragePooling2D, Activation, GlobalAveragePooling2D, Concatenate, BatchNormalization, MaxPooling2D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers.merge import concatenate
from tensorflow.keras.models import load_model
from imgaug import augmenters as iaa

num_classes = 200

## Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -qq 'tiny-imagenet-200.zip'

--2021-12-21 13:16:52--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  36.4MB/s    in 8.2s    

2021-12-21 13:17:00 (29.0 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [ ]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)

In [ ]:
train_data = ImageDataGenerator(rescale= 1./255)

valid_data = ImageDataGenerator(rescale=1./255)

train_generator = train_data.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                 target_size=(32, 32), 
                                                 color_mode='rgb', 
                                                 batch_size=256, 
                                                 class_mode='categorical', 
                                                 shuffle=True, 
                                                 seed=42)

validation_generator = valid_data.flow_from_dataframe(val_data, 
                                                      directory='./tiny-imagenet-200/val/images/', 
                                                      x_col='File', 
                                                      y_col='Class', 
                                                      target_size=(64, 64),
                                                      color_mode='rgb', 
                                                      class_mode='categorical', 
                                                      batch_size=256, 
                                                      shuffle=True, 
                                                      seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


## Original Paper's Model 1

In [ ]:
def space_to_depth(x):
    return tf.nn.space_to_depth(x, block_size=2)

In [ ]:
input = Input(shape=(None, None, 3))

# Block 1

layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer1 = BatchNormalization(name='norm_1')(layer1)
layer1 = Activation("relu")(layer1)

layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer2 = BatchNormalization(name='norm_2')(layer2)
layer2 = Activation("relu")(layer2)

layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer2)
layer3 = BatchNormalization(name='norm_3')(layer3)
layer3 = Activation("relu")(layer3)

layer4 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer3)
layer4 = BatchNormalization(name='norm_4')(layer4)
layer4 = Activation("relu")(layer4)

layer5 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(layer4)
layer5 = BatchNormalization(name='norm_5')(layer5)
layer5 = Activation("relu")(layer5)

layer6 = MaxPooling2D(pool_size=(2, 2))(layer5)

skip_connection_1 = layer6

# Block 2

layer7 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer6)
layer7 = BatchNormalization(name='norm_7')(layer7)
layer7 = Activation("relu")(layer7)

layer8 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer7)
layer8 = BatchNormalization(name='norm_8')(layer8)
layer8 = Activation("relu")(layer8)

layer9 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer8)
layer9 = BatchNormalization(name='norm_9')(layer9)
layer9 = Activation("relu")(layer9)

layer10 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_10', use_bias=False)(layer9)
layer10 = BatchNormalization(name='norm_10')(layer10)
layer10 = Activation("relu")(layer10)

layer11 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer10)
layer11 = BatchNormalization(name='norm_11')(layer11)
layer11 = Activation("relu")(layer11)

layer12 = MaxPooling2D(pool_size=(2, 2))(layer11)

skip_connection_1 = Lambda(space_to_depth)(skip_connection_1)

layer13 = concatenate([skip_connection_1, layer12])

skip_connection_2 = layer13

# Block 3

layer14 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer13)
layer14 = BatchNormalization(name='norm_14')(layer14)
layer14 = Activation("relu")(layer14)

layer15 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_15', use_bias=False)(layer14)
layer15 = BatchNormalization(name='norm_15')(layer15)
layer15 = Activation("relu")(layer15)

layer16 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(layer15)
layer16 = BatchNormalization(name='norm_16')(layer16)
layer16 = Activation("relu")(layer16)

layer17 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_17', use_bias=False)(layer16)
layer17 = BatchNormalization(name='norm_17')(layer17)
layer17 = Activation("relu")(layer17)

layer18 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(layer17)
layer18 = BatchNormalization(name='norm_18')(layer18)
layer18 = Activation("relu")(layer18)

layer19 = MaxPooling2D(pool_size=(2, 2))(layer18)

skip_connection_2 = Lambda(space_to_depth)(skip_connection_2)

layer20 = concatenate([skip_connection_2, layer19])

layer21 = Conv2D(num_classes, (1,1), name='conv_21', use_bias=False)(layer20)
layer21 = BatchNormalization(name='norm_21')(layer21)

layer22 = GlobalAveragePooling2D(data_format=None)(layer21)

layer23 = Activation('softmax')(layer22)

output = layer23

In [ ]:
model = tf.keras.models.Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv_1 (Conv2D)                (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 norm_1 (BatchNormalization)    (None, None, None,   128         ['conv_1[0][0]']                 
                                32)                                                         

In [ ]:
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Training

### Epoch 1-15 
### 32x32 lr_reducer

In [ ]:
history1 = model.fit(train_generator, 
                     epochs=15, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator, 
                     callbacks=[lr_reducer])

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_1.h5') 
df = pd.DataFrame(history1.history)
df.to_csv('history1.csv')
from google.colab import files
files.download('history1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Epoch 15-45
### 64x64 

In [ ]:
train_generator = train_data.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                    target_size=(64, 64), 
                                                    color_mode='rgb', 
                                                    batch_size=64, 
                                                    class_mode='categorical', 
                                                    shuffle=True, 
                                                    seed=42)

validation_generator = valid_data.flow_from_dataframe(val_data, 
                                                         directory='./tiny-imagenet-200/val/images/', 
                                                         x_col='File', 
                                                         y_col='Class', 
                                                         target_size=(64, 64),
                                                         color_mode='rgb', 
                                                         class_mode='categorical', 
                                                         batch_size=64, 
                                                         shuffle=True, 
                                                         seed=42)

In [ ]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_1.h5')

history2 = model.fit(train_generator, 
                     epochs=30, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator)

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_2.h5') 
df = pd.DataFrame(history2.history)
df.to_csv('history2.csv')
from google.colab import files
files.download('history2.csv')

Epoch 1/30
200/200 [==============================] - 83s 411ms/step - loss: 3.0180 - accuracy: 0.3328 - val_loss: 4.0202 - val_accuracy: 0.1863
Epoch 2/30
200/200 [==============================] - 82s 410ms/step - loss: 2.8010 - accuracy: 0.3701 - val_loss: 4.1471 - val_accuracy: 0.1705
Epoch 3/30
200/200 [==============================] - 82s 410ms/step - loss: 2.6619 - accuracy: 0.3998 - val_loss: 3.1779 - val_accuracy: 0.2908
Epoch 4/30
200/200 [==============================] - 82s 411ms/step - loss: 2.5500 - accuracy: 0.4203 - val_loss: 3.1909 - val_accuracy: 0.2846
Epoch 5/30
200/200 [==============================] - 84s 419ms/step - loss: 2.4969 - accuracy: 0.4224 - val_loss: 2.8337 - val_accuracy: 0.3525
Epoch 6/30
200/200 [==============================] - 82s 410ms/step - loss: 2.4087 - accuracy: 0.4495 - val_loss: 3.1104 - val_accuracy: 0.3062
Epoch 7/30
200/200 [==============================] - 82s 410ms/step - loss: 2.3503 - accuracy: 0.4617 - val_loss: 2.8888 - val_ac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Epoch 45-55
### 16x16

In [ ]:
train_generator = train_data.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                 target_size=(16, 16), 
                                                 color_mode='rgb', 
                                                 batch_size=256, 
                                                 class_mode='categorical', 
                                                 shuffle=True, 
                                                 seed=42)

validation_generator = valid_data.flow_from_dataframe(val_data, 
                                                      directory='./tiny-imagenet-200/val/images/', 
                                                      x_col='File', 
                                                      y_col='Class', 
                                                      target_size=(64, 64),
                                                      color_mode='rgb', 
                                                      class_mode='categorical', 
                                                      batch_size=256, 
                                                      shuffle=True, 
                                                      seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [ ]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_2.h5')

history3 = model.fit(train_generator, 
                     epochs=10, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator)

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_3.h5')

df = pd.DataFrame(history3.history)
df.to_csv('history3.csv')
from google.colab import files
files.download('history3.csv')

Epoch 1/10
200/200 [==============================] - 41s 200ms/step - loss: 3.5624 - accuracy: 0.2415 - val_loss: 4.2976 - val_accuracy: 0.1584
Epoch 2/10
200/200 [==============================] - 39s 195ms/step - loss: 2.7443 - accuracy: 0.3598 - val_loss: 4.1455 - val_accuracy: 0.1516
Epoch 3/10
200/200 [==============================] - 39s 196ms/step - loss: 2.3815 - accuracy: 0.4362 - val_loss: 4.1954 - val_accuracy: 0.1537
Epoch 4/10
200/200 [==============================] - 39s 195ms/step - loss: 2.0877 - accuracy: 0.4990 - val_loss: 4.5643 - val_accuracy: 0.1185
Epoch 5/10
200/200 [==============================] - 39s 195ms/step - loss: 1.7892 - accuracy: 0.5731 - val_loss: 4.5667 - val_accuracy: 0.1270
Epoch 6/10
200/200 [==============================] - 39s 195ms/step - loss: 1.4989 - accuracy: 0.6490 - val_loss: 4.6892 - val_accuracy: 0.1210
Epoch 7/10
200/200 [==============================] - 39s 195ms/step - loss: 1.2376 - accuracy: 0.7188 - val_loss: 4.7329 - val_ac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Epoch 55-85
### 64x64



In [ ]:
train_generator = train_data.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                 target_size=(64, 64), 
                                                 color_mode='rgb', 
                                                 batch_size=64, 
                                                 class_mode='categorical', 
                                                 shuffle=True, 
                                                 seed=42)

validation_generator = valid_data.flow_from_dataframe(val_data, 
                                                      directory='./tiny-imagenet-200/val/images/', 
                                                      x_col='File', 
                                                      y_col='Class', 
                                                      target_size=(64, 64),
                                                      color_mode='rgb', 
                                                      class_mode='categorical', 
                                                      batch_size=64, 
                                                      shuffle=True, 
                                                      seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [ ]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_3.h5')

history4 = model.fit(train_generator, 
                     epochs=30, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator)

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_4.h5')

df = pd.DataFrame(history4.history)
df.to_csv('history4.csv')
from google.colab import files
files.download('history4.csv')

Epoch 1/30
200/200 [==============================] - 83s 410ms/step - loss: 1.9384 - accuracy: 0.5340 - val_loss: 3.0396 - val_accuracy: 0.3360
Epoch 2/30
200/200 [==============================] - 82s 411ms/step - loss: 1.6575 - accuracy: 0.5948 - val_loss: 2.5075 - val_accuracy: 0.4179
Epoch 3/30
200/200 [==============================] - 82s 409ms/step - loss: 1.5708 - accuracy: 0.6151 - val_loss: 2.5155 - val_accuracy: 0.4213
Epoch 4/30
200/200 [==============================] - 82s 411ms/step - loss: 1.4715 - accuracy: 0.6436 - val_loss: 2.5459 - val_accuracy: 0.4223
Epoch 5/30
200/200 [==============================] - 82s 411ms/step - loss: 1.4378 - accuracy: 0.6510 - val_loss: 2.2261 - val_accuracy: 0.4761
Epoch 6/30
200/200 [==============================] - 82s 410ms/step - loss: 1.3915 - accuracy: 0.6656 - val_loss: 2.3371 - val_accuracy: 0.4511
Epoch 7/30
200/200 [==============================] - 82s 410ms/step - loss: 1.3603 - accuracy: 0.6739 - val_loss: 2.2054 - val_ac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Image Augmentation

In [ ]:
seq = iaa.SomeOf((0, None), 
                 [
                  iaa.Affine(scale=(0.5, 1.0)), 
                  iaa.CoarseDropout((0.0, 0.1), size_percent=(0.06, 0.08)), 
                  iaa.Affine(rotate=30),
                  iaa.AdditiveGaussianNoise(scale=0.1*255), 
                  iaa.CropAndPad(percent=(-0.2, 0.2))
                  ])

train_data = ImageDataGenerator(preprocessing_function=seq.augment_image, rescale=1./255)

valid_data = ImageDataGenerator(rescale=1./255)

### Epoch 85-235

In [ ]:
train_generator = train_data.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                 target_size=(64, 64), 
                                                 color_mode='rgb', 
                                                 batch_size=64, 
                                                 class_mode='categorical', 
                                                 shuffle=True, 
                                                 seed=42)

validation_generator = valid_data.flow_from_dataframe(val_data, 
                                                      directory='./tiny-imagenet-200/val/images/', 
                                                      x_col='File', 
                                                      y_col='Class', 
                                                      target_size=(64, 64),
                                                      color_mode='rgb', 
                                                      class_mode='categorical', 
                                                      batch_size=64, 
                                                      shuffle=True, 
                                                      seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [ ]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_4.h5')

history5 = model.fit(train_generator, 
                     epochs=150, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator)

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_5.h5')

df = pd.DataFrame(history5.history)
df.to_csv('history5.csv')
from google.colab import files
files.download('history5.csv')

Epoch 1/150
200/200 [==============================] - 107s 528ms/step - loss: 2.7863 - accuracy: 0.3698 - val_loss: 3.2267 - val_accuracy: 0.3633
Epoch 2/150
200/200 [==============================] - 98s 492ms/step - loss: 2.4923 - accuracy: 0.4187 - val_loss: 2.6426 - val_accuracy: 0.4222
Epoch 3/150
200/200 [==============================] - 99s 493ms/step - loss: 2.4221 - accuracy: 0.4326 - val_loss: 2.8214 - val_accuracy: 0.3895
Epoch 4/150
200/200 [==============================] - 98s 490ms/step - loss: 2.3390 - accuracy: 0.4473 - val_loss: 2.4966 - val_accuracy: 0.4397
Epoch 5/150
200/200 [==============================] - 98s 491ms/step - loss: 2.3021 - accuracy: 0.4607 - val_loss: 2.5575 - val_accuracy: 0.4360
Epoch 6/150
200/200 [==============================] - 99s 493ms/step - loss: 2.2496 - accuracy: 0.4671 - val_loss: 2.7623 - val_accuracy: 0.4052
Epoch 7/150
200/200 [==============================] - 99s 494ms/step - loss: 2.2320 - accuracy: 0.4678 - val_loss: 2.6349 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Model Checkpoint

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/{epoch:02d}-{val_accuracy:.2f}.hdf5', 
                                                monitor='val_accuracy', 
                                                save_best_only=True, 
                                                save_weights_only=False, 
                                                mode='auto')

In [ ]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_5.h5')

history17 = model.fit(train_generator, 
                      epochs=10, 
                      steps_per_epoch=200, 
                      validation_data=validation_generator,  
                      callbacks=[lr_reducer,checkpoint])

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_6.h5')

Epoch 1/10
200/200 [==============================] - 83s 413ms/step - loss: 1.6926 - accuracy: 0.5910 - val_loss: 2.2583 - val_accuracy: 0.4817 - lr: 0.0010
Epoch 2/10
200/200 [==============================] - 81s 407ms/step - loss: 1.0201 - accuracy: 0.7290 - val_loss: 2.2879 - val_accuracy: 0.4756 - lr: 0.0010
Epoch 3/10
200/200 [==============================] - 81s 406ms/step - loss: 0.7657 - accuracy: 0.7990 - val_loss: 2.2554 - val_accuracy: 0.4766 - lr: 0.0010
Epoch 4/10
200/200 [==============================] - 81s 405ms/step - loss: 0.5780 - accuracy: 0.8522 - val_loss: 2.3007 - val_accuracy: 0.4706 - lr: 0.0010
Epoch 5/10
200/200 [==============================] - 81s 405ms/step - loss: 0.4536 - accuracy: 0.8899 - val_loss: 2.3510 - val_accuracy: 0.4608 - lr: 0.0010
Epoch 6/10
200/200 [==============================] - 81s 405ms/step - loss: 0.3495 - accuracy: 0.9227 - val_loss: 2.3236 - val_accuracy: 0.4703 - lr: 0.0010
Epoch 7/10
200/200 [==============================] 

### Evaluation

In [ ]:
valid_data = ImageDataGenerator(rescale=1./255)
validation_generator = valid_data.flow_from_dataframe(val_data, 
                                                      directory='./tiny-imagenet-200/val/images/', 
                                                      x_col='File', 
                                                      y_col='Class', 
                                                      target_size=(64, 64),
                                                      color_mode='rgb', 
                                                      class_mode='categorical', 
                                                      batch_size=64, 
                                                      shuffle=True, 
                                                      seed=42)

Found 10000 validated image filenames belonging to 200 classes.


In [ ]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_1/Model1_5.h5')

validation_steps_per_epoch = np.math.ceil(validation_generator.samples / validation_generator.batch_size)

score = model.evaluate(validation_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

157/157 [==============================] - 27s 111ms/step - loss: 2.1563 - accuracy: 0.5377
Test loss: 2.1563007831573486
Test accuracy: 0.5376999974250793
